# Linear Regression - Least Squares  

-------------  

_Author: Khal Makhoul, W.P.G.Peterson_  

## Project Guide
----------------  
- [Project Overview](#overview)
- [Introduction and Review](#intro)
- [Data Exploration](#data)  
- [Coding Linear Regression](#code)

<a id = "overview"></a>
## Project Overview
----------
#### EXPECTED TIME 2 HRS

This assignment will test your ability to code your own version of least squares regression in `Python`. After a brief review of some of the content from the lecture you will be asked to create a number of functions that will eventually be able to read in raw data to `Pandas` and perform a least squares regression on a subset of that data.  

This will include:  
- Calculating least squares weights
- reading data on dist to return `Pandas` DataFrame  
- select data by column  
- implement column cutoffs  

** Motivation**: Least squares regression offer a way to build a closed-form and interpretable model.  

**Objectives**: This assignment will:
- Test `Python` and `Pandas` competency
- Ensure understanding of the mathematical foundations behind least squares regression  

**Problem**: Using housing data, we will attempt to predict house price using living area with a regression model.  

**Data**: Our data today comes from [Kaggle's House Prices Dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).  

See above link for Description of data from `Kaggle`.  

<a id = "intro"></a>
### Introduction and Review

As long as a few basic assumptions are fulfilled, linear regression using least squares is solvable exactly, without requiring approximation. 

This means that the equations presented in the week 1 lectures can be adapted directly to `Python` code, making this good practice both for using `Python` and translating an "algorithm" to code.

We will use the matrix version of the least squares solution presented in lecture to derive the desired result. As a reminder, this expresses the least squares coefficients $w_{LS}$ as a vector, and calculates that vector as a function of $X$, the matrix of inputs, and $y$, the vector of outputs from the training set:

$$w_{LS} = (X^T X)^{−1}X^T y,$$

where $w_{LS}$ refers to the vector of weights we are trying to find, $X$ is the matrix of inputs, and $y$ is the output vector. 

In this equation, $X$ is always defined to have a vector of $1$ values as its first column. In other words, even when there is only one input value for each data point, $X$ takes the form:

$$
X = \begin{bmatrix}
1 \  x_{11}  \\
1 \  x_{21}  \\
\vdots \ \vdots \\
1 \ x_{n1}
\end{bmatrix} 
$$

For two inputs per data point, $X$ will take this form:
 
$$
X = \begin{bmatrix}
1 \  x_{11} \  x_{12} \\
1 \  x_{21} \  x_{22} \\
\vdots \ \vdots \\
1 \ x_{n1} \  x_{n2}
\end{bmatrix} 
$$




Please refer to lecture notes for additional context.  
<a id = "data"></a>
### Data Exploration

Before coding an algorithm, we will take a look at our data using `Python`'s `pandas`. For visualizations we'll use `matplotlib`. Familiarity with these modules will serve you well. The following cells include comments to explain the purpose of each step.

In [ ]:
### This cell imports the necessary modules and sets a few plotting parameters for display

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)

In [ ]:
### Read in the data
tr_path = 'data/Assignment 1 - datasets/train.csv'
test_path = 'data/Assignment 1 - datasets/test.csv'
data = pd.read_csv(tr_path)

In [ ]:
### The .head() function shows the first few lines of data for perspecitve
data.head()

In [ ]:
### Lists column names
#data.columns
data.columns.size

In [ ]:
### GRADED
### How many columns are in `data`?
### assign int answer to ans1
### YOUR ANSWER BELOW

ans1 = data.columns.size

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Visualizations

In [ ]:
### We can plot the data as follows
### Price v. living area
### with matplotlib

Y = data['SalePrice']
X = data['GrLivArea']

plt.scatter(X, Y, marker = "x")

### Annotations
plt.title("Sales Price vs. Living Area (excl. basement)")
plt.xlabel("GrLivArea")
plt.ylabel("SalePrice");

In [ ]:
### price v. year
### Using Pandas

data.plot('YearBuilt', 'SalePrice', kind = 'scatter', marker = 'x');

In [ ]:
### GRADED
### Given the above graphs, it appears there is a:  
### True) positive correlation between the variables
### False) negative correlation between the variables
### Assign boolean corresponding to choice to ans1
### YOUR ANSWER BELOW

ans1 = True

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


### Submission Instructions

You will have to ensure that the function names match the examples provided.

The code will be automatically graded by a script that will take your code as input and execute it. In order for the grading script to work properly, you must follow the naming conventions in this assignment stub.  

<a id = "code"></a>
### Coding Linear Regression
Given the equation above for $w_{LS}$, we know all we need to in order to solve a linear regression. Coding out the steps in `Python`, we will complete the process in several steps.

#### Matrix Operations
Below is an example of a function that takes the inverse of a matrix. The `numpy` module is used, and all the function does is call the `numpy` function `np.linalg.inv()`. Though simple, this can be used as a template for a few good coding practices:

* Name functions and parameters descriptively
* Use underscores _ to separate words in variable/function names (snake_case, **NOT** PascalCase or camelCase)
* In functions and classes, include a docstring between triple quotes 

In [ ]:
def get_deternminant(mat):
    """
    Calculate and return the determinant of a matrix
    Argument -- a matrix
    """
    #for 2x2 matrix
    if len(mat) == 2:
        return (mat[0][0] * mat[1][1]) - (mat[0][1] * mat[1][0])

    determinant = 0
    for c in range(len(mat)):
        determinant += ((-1)**c) * mat[0][c] * get_deternminant([row[:c] + row[c + 1:] for row in (mat[:0] + mat[1:])])
    return determinant

def inverse_of_matrix(mat):
    """Calculate and return the multiplicative inverse of a matrix.
    
    Positional argument:
        mat -- a matrix to invert
    
    Example:
        sample_matrix = [[1, 2], [3, 4]]
        the_inverse = inverse_of_matrix(sample_matrix)
        
    Notes:
        Part of the code used from https://stackoverflow.com/questions/32114054/matrix-inversion-without-numpy. 
        Changes done to make the routine less complex and to standarize input and output 
    """
    if isinstance(mat, pd.DataFrame) :
        mat = mat.values
      
    determinant = get_deternminant(mat)
    #for 2x2 matrix:
    if len(mat) == 2:
        return np.array([[mat[1][1]/determinant, -1*mat[0][1]/determinant],[-1*mat[1][0]/determinant, mat[0][0]/determinant]])

    #find matrix of cofactors
    cofactors = []
    for r in range(len(mat)):
        cofactorRow = []
        for c in range(len(mat)):
            #minor = get_minor(m,r,c)
            minor = [row[:c] + row[c + 1:] for row in (mat[:r] + mat[r + 1:])]
            cofactorRow.append(((-1)**(r+c)) * get_deternminant(minor))
        cofactors.append(cofactorRow)
    #cofactors = transpose_matrix(cofactors)
    cofactors = [[cofactors[j][i] for j in range(len(cofactors))] for i in range(len(cofactors[0]))]
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return np.array(cofactors)

print("test",inverse_of_matrix([[1,2],[3,4]]),"\n")
print("From Data:\n", inverse_of_matrix(data.iloc[:2,:2]))

In [ ]:
### GRADED
### Build a function that takes as input a matrix
### return the inverse of that matrix
### assign function to "inverse_of_matrix"
### YOUR ANSWER BELOW

def inverse_of_matrix_easy(mat):
    """Calculate and return the multiplicative inverse of a matrix.
    
    Positional argument:
        mat -- a square matrix to invert
    
    Example:
        sample_matrix = [[1, 2], [3, 4]]
        the_inverse = inverse_of_matrix(sample_matrix)
        
    Requirements:
        This function depends on the numpy function `numpy.linalg.inv`. 
    """
    matrix_inverse = np.linalg.inv(mat)
    return matrix_inverse

### Testing function:

#print(type([[1,2],[3,4]]))
print("test",inverse_of_matrix_easy([[1,2],[3,4]]),"\n")
#print(type(inverse_of_matrix([[1,2],[3,4]])))

#print(type(data.iloc[:2,:2]))
print("From Data:\n", inverse_of_matrix_easy(data.iloc[:2,:2]))
#print(type(inverse_of_matrix(data.iloc[:2,:2])))

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Q1: Read Data

In [ ]:
### GRADED
### In order to create any model it is necessary to read in data
### Build a function called "read_to_df" that takes the file_path of a .csv file.
### Use a pandas functions appropriate for .csv files to turn that path into a DataFrame
### Use pandas function defaults for reading in file
### Return that DataFrame
### Grade will be determined by whether or not the returned item is of type "DataFrame" and
### if the dimensions are correct
### YOUR ANSWER BELOW
import pandas as pd
def read_to_df(file_path):
    """Read on-disk data and return a dataframe."""
    data = pd.read_csv(file_path)
    
    return data

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Q2: Select by Columns

In [ ]:
### GRADED
### Build a function called "select_columns"
### As inputs, take a DataFrame and a *list* of column names.
### Return a DataFrame that only has the columns specified in the list of column names
### Grading will check type of object, dimensions of object, and column names
### YOUR ANSWER BELOW

def select_columns(data_frame, column_names):
    """Return a subset of a data frame by column names.

    Positional arguments:
        data_frame -- a pandas DataFrame object
        column_names -- a list of column names to select

    Example:
        data = read_to_df('train.csv')
        selected_columns = ['SalePrice', 'GrLivArea', 'YearBuilt']
        sub_df = select_columns(data, selected_columns)
    """

    return data_frame[column_names]

data = read_to_df('data/Assignment 1 - datasets/train.csv')
print(data.index)
print(data.index.values)
selected_columns = ['SalePrice', 'GrLivArea', 'YearBuilt']
#print(type(selected_columns))
sub_df = select_columns(data, selected_columns)
print(sub_df.head())
print(type(sub_df))

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Q2a:

In [ ]:
### GRADED
### For a `Pandas` DataFrame named `df`, the names of columns may be accessed by the:
### `df.columns` attribute.
### The names of the rows may be accessed by the `df.<ans1>` attribute
### to ans1 assign a string that when placed after `df.` will return the row names
### of a DataFrame
### YOUR ANSWER BELOW

ans1 = 'index.values'

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Q3: Subset Data by Value

In [ ]:
### GRADED
### Build a function called "column_cutoff"
### As inputs, accept a Pandas Dataframe and a list of tuples.
### Tuples in format (column_name, min_value, max_value)
### Return a DataFrame which excludes rows where the value in specified column exceeds "max_value"
### or is less than "min_value"
### ### NB: DO NOT remove rows if the column value is equal to the min/max value
### YOUR ANSWER BELOW

def column_cutoff(data_frame, cutoffs):
    """Subset data frame by cutting off limits on column values.
    
    Positional arguments:
        data -- pandas DataFrame object
        cutoffs -- list of tuples in the format: 
        (column_name, min_value, max_value)
        
    Example:
        data_frame = read_into_data_frame('train.csv')
        # Remove data points with SalePrice < $50,000
        # Remove data points with GrLiveAre > 4,000 square feet
        cutoffs = [('SalePrice', 50000, 1e10), ('GrLivArea', 0, 4000)]
        selected_data = column_cutoff(data_frame, cutoffs)
        
    Requirements:
        This function does not consider empty cutoffs tuples 
    """
    
    frames = [data_frame.loc[(data_frame[a] >= b) & (data_frame[a] <= c)] for a, b, c in cutoffs]
    df_filtered = frames[0]
    for i in range(len(frames) - 1):
        df_filtered = pd.merge(df_filtered, frames[i+1], how = 'inner')
    
    return df_filtered

data = read_to_df('data/Assignment 1 - datasets/train.csv')
cutoffs = [('GrLivArea', 0, 4000),('SalePrice', 50000, 10000000000)]
selected_data = column_cutoff(data, cutoffs)
print(len(selected_data))


In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


Next you'll implement the equation above for $w_{LS}$ using the inverse matrix function.  
$$w_{LS} = (X^T X)^{−1}X^T y,$$

#### Q4: Least Squares

In [ ]:
### GRADED
### Build a function  called "least_squares_weights"
### take as input two matricies corresponding to the X inputs and y target
### assume the matricies are of the correct dimensions

### Step 1: ensure that the number of rows of each matrix is greater than or equal to the number
### of columns.
### ### If not, transpose the matricies.
### In particular, the y input should end up as a n-by-1 matrix, and the x input as a n-by-p matrix

### Step 2: *prepend* an n-by-1 column of ones to the input_x matrix

### Step 3: Use the above equation to calculate the least squares weights.

### NB: `.shape`, `np.matmul`, `np.linalg.inv`, `np.ones` and `np.transpose` will be valuable.
### If those above functions are used, the weights should be accessable as below:  
### weights = least_squares_weights(train_x, train_y)
### weight1 = weights[0][0]; weight2 = weights[1][0];... weight<n+1> = weights[n][0]

### YOUR ANSWER BELOW


def least_squares_weights(input_x, target_y):
    """ add desc """
    #Step 1
    y_shape = target_y.shape
    y = target_y.transpose() if y_shape[1] >= y_shape[0] else target_y
    
    x_shape = input_x.shape
    X = input_x.transpose() if x_shape[1] >= x_shape[0] else input_x
       
    #Step2
    n = len(X)
    X = np.hstack((np.ones((n,1), dtype=np.float64), X))
    
    #Step3
    w_LS = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
    
    return w_LS

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
### GRADED
### Why, in the function  above, is it necessary to prepend a column of ones
### 'a') To re-shape the matrix
### 'b') To create an intercept term
### 'c') It isn't needed, it's just meant to be confusing
### 'd') As a way to make sure the weights turn out positive
### Assign the character asociated with your choice as a string to ans1
### YOUR ANSWER BELOW

ans1 = 'b'

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Testing on Real Data

Now that we have code to read the data and perform matrix operations, we can put it all together to perform linear regression on a data set of our choosing.  

If your functions above are defined correctly, the following two cells should run without error.

In [ ]:
df = read_to_df(tr_path)
df_sub = select_columns(df, ['SalePrice', 'GrLivArea', 'YearBuilt'])

cutoffs = [('SalePrice', 50000, 1e10), ('GrLivArea', 0, 4000)]
df_sub_cutoff = column_cutoff(df_sub, cutoffs)

X = df_sub_cutoff['GrLivArea'].values
Y = df_sub_cutoff['SalePrice'].values

### reshaping for input into function
training_y = np.array([Y])
training_x = np.array([X])

weights = least_squares_weights(training_x, training_y)
print(weights)

In [ ]:
max_X = np.max(X) + 500
min_X = np.min(X) - 500

### Choose points evenly spaced between min_x in max_x
reg_x = np.linspace(min_X, max_X, 1000)

### Use the equation for our line to calculate y values
reg_y = weights[0][0] + weights[1][0] * reg_x

plt.plot(reg_x, reg_y, color='#58b970', label='Regression Line')
plt.scatter(X, Y, c='k', label='Data')

plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.legend()
plt.show()

#### Model Evalutaion Intro
Further lessons will discuss model evaluation scores in more detail, quickly, here we will calculate root mean squared errors with our calculated weights

In [ ]:
### GRADED
### True or False
### The Root Mean Square Error is in the same units as the data
### assign boolean response to ans1
### YOUR ANSWER BELOW

ans1 = True

In [ ]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


#### Calculating RMSE

In [ ]:
rmse = 0

b0 = weights[0][0]
b1 = weights[1][0]

for i in range(len(Y)):
    y_pred = b0 + b1 * X[i]
    rmse += (Y[i] - y_pred) ** 2
rmse = np.sqrt(rmse/len(Y))
print(rmse)

#### Calculating $R^2$

In [ ]:
ss_t = 0
ss_r = 0

mean_y = np.mean(Y)

for i in range(len(Y)):
    y_pred = b0 + b1 * X[i]
    ss_t += (Y[i] - mean_y) ** 2
    ss_r += (Y[i] - y_pred) ** 2
r2 = 1 - (ss_r/ss_t)

print(r2)

### sklearn implementation

While it is useful to build and program our model from scratch, this course will also introduce how to use conventional methods to fit each model. In particular, we will be using the `scikit-learn` module (also called `sklearn`.)  

Check to see how close your answers are!

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

### sklearn requires a 2-dimensional X and 1 dimensional y. The below yeilds shapes of:
### skl_X = (n,1); skl_Y = (n,)
skl_X = df_sub_cutoff[['GrLivArea']]
skl_Y = df_sub_cutoff['SalePrice']

lr.fit(skl_X,skl_Y)
print("Intercept:", lr.intercept_)
print("Coefficient:", lr.coef_)